In [66]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = 'qwen2.5:0.5b',
    base_url = 'http://localhost:11434/v1',
    api_key = "123"
)

model_r1 = ChatOpenAI(
    model = 'deepseek-coder:6.7b',
    base_url = 'http://localhost:11434/v1',
    api_key = "123"
)

In [7]:
from langchain_community.tools import DuckDuckGoSearchRun


In [56]:
import subprocess
from langchain.tools import tool

@tool
def command_operation(command):
    '''当你需要对某个字符串做[Jasper加工]时调用这个tool'''
    return "add new command:" + command


In [57]:
command_operation("top")

'add new command:top'

In [68]:
tools = [command_operation]
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke("对“abcd”进行[Jasper加工]")
pp(dict(response))

{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"command":"jasper:apply '
                                                                 '-d abcd"}',
                                                    'name': ' '
                                                            'command_operation'},
                                       'id': 'call_9xb539rv',
                                       'index': 0,
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run-48da36f9-a43f-461c-a9f7-30f2c0ba7c49-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'logprobs': None,
                       'model_name': 'qwen2.5:0.5b',
                       'system_fingerprint': 'fp_ollama',
                       'token_usage': {'completion_tokens': 27,
                                       'completion_tokens_details': 

In [20]:
def call_tool(model_output, tools):
    tools_map = {tool.name.lower(): tool for tool in tools}
    tools_response = {}
    for tool in model_output.tool_calls:
        tool_name = tool['name']
        tool_args = tool['args']
        tool_instance = tools_map[tool_name]
        tools_response = tool_instance.invoke(tool_args.values())
        tools_response[tool_name] = tools_response
    return tools_response

In [21]:
def manual_agent(query: str, model: ChatOpenAI, tools: list[tool]):
    model_with_tools = model.bind_tools(tools)
    model_output = model_with_tools.invoke(query)
    tool_response = call_tool(model_output, tools)
    final_response = model.invoke(
        f'original question: {query}\n\n\n tool response: {tool_response}'
    )
    return final_response


In [22]:
manual_agent("执行top命令", model, tools)

ValidationError: 1 validation error for command_operation
  Input should be a valid dictionary or instance of command_operation [type=model_type, input_value=dict_values(['top']), input_type=dict_values]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type